# Example in Class
started on 20-Mar-2019

Example of deriving the equations of motion of an inverted pendulum on a cart using Hamilton's equations

In [1]:
using Pkg
Pkg.activate(".")

# uncomment the next line the first time this sheet is run
#Pkg.add("SymPy")

using SymPy

In [2]:
m,M,g,l,J= symbols("m,M,g,l,J_G", positive=true)

(m, M, g, l, J_G)

In [3]:
t,τ, f = symbols("t, tau, f")
x = SymFunction("x")(t)
θ = SymFunction("theta")(t)
;

In [4]:
ihat= [1,0,0]
jhat = [0,1,0]
khat= [0,0,1]
;

In [5]:
rG = x*ihat + l*(sin(θ)*ihat + cos(θ)*jhat )

3-element Array{Sym,1}:
 l*sin(theta(t)) + x(t)
        l*cos(theta(t))
                      0

In [6]:
T = 1//2*M*diff(x,t)^2 + 1//2*m*transpose(diff(rG,t))*diff(rG,t) + 1//2*J*diff(θ,t)^2 |> simplify

                  2               2                      2                    
    /d           \      /d       \     2   /d           \                     
J_G*|--(theta(t))|    M*|--(x(t))|    l *m*|--(theta(t))|                     
    \dt          /      \dt      /         \dt          /                     
------------------- + ------------- + -------------------- + l*m*cos(theta(t))
         2                  2                  2                              

                                     2
                           /d       \ 
                         m*|--(x(t))| 
 d            d            \dt      / 
*--(theta(t))*--(x(t)) + -------------
 dt           dt               2      

In [7]:
V = m*g*rG[2]

g*l*m*cos(theta(t))

In [8]:
p1,p2 = symbols("p1,p2")

(p1, p2)

In [9]:
L = T-V

                  2               2                                           
    /d           \      /d       \                           2   /d           
J_G*|--(theta(t))|    M*|--(x(t))|                          l *m*|--(theta(t))
    \dt          /      \dt      /                               \dt          
------------------- + ------------- - g*l*m*cos(theta(t)) + ------------------
         2                  2                                        2        

 2                                                         2
\                                                /d       \ 
|                                              m*|--(x(t))| 
/                      d            d            \dt      / 
-- + l*m*cos(theta(t))*--(theta(t))*--(x(t)) + -------------
                       dt           dt               2      

In [10]:
eq1 = [Eq(p1, diff(L,diff(x,t))),
    Eq(p2, diff(L,diff(θ,t))) ]

2-element Array{Sym,1}:
           Eq(p1, M*Derivative(x(t), t) + l*m*cos(theta(t))*Derivative(theta(t), t) + m*Derivative(x(t), t))
 Eq(p2, J_G*Derivative(theta(t), t) + l^2*m*Derivative(theta(t), t) + l*m*cos(theta(t))*Derivative(x(t), t))

In [11]:
solp =  solve(eq1, diff([x,θ], t) )

Dict{Sym,Sym} with 2 entries:
  Derivative(theta(t), t) => (l*m*p1*cos(theta(t)) - p2*(M + m))/(l^2*m^2*cos(t…
  Derivative(x(t), t)     => (l*m*p2*cos(theta(t)) - p1*(J_G + l^2*m))/(l^2*m^2…

Example ended here in class, next class we will finish this up

Starting now (22-Mar), let's build the Hamiltonian

In [12]:
H = T + V |> subs(solp) |> simplify

                                                             2                
                                     2                 J_G*p1         3  2    
J_G*M*g*l*m*cos(theta(t)) + J_G*g*l*m *cos(theta(t)) + ------- + M*g*l *m *cos
                                                          2                   
------------------------------------------------------------------------------
                                                                              
                                                                        J_G*M 

                 2                                                     2     2
             M*p2       3  3    3                3  3                 l *m*p1 
(theta(t)) + ----- - g*l *m *cos (theta(t)) + g*l *m *cos(theta(t)) + --------
               2                                                         2    
------------------------------------------------------------------------------
             2      2  2    2                      

In [13]:
pdot = [ - diff(H,q) for q in [x,θ] ] + [f, τ]

2-element Array{Sym,1}:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  f
 2*l^2*m^2*(J_G*M*g*l*m*cos(theta(t)) + J_G*g*l*m^2*cos(theta(t)) + J_G*p1^2/2 + M*g*l^3*m^2*cos(theta(t)) + M*p2^2/2 - g*l^3*m^3*cos(theta(t))^3 + g*l^3*m^3*cos(theta(t)) + l^2*m*p1^2/2 - l*m*p1*p2*cos(theta(t)) + m*p2^2/2)*sin(theta(t))*cos(theta(t))/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(theta(t))^2)^2 + tau - (-J_G*M*g*l*m*sin(theta(t)) - J_G*g*l*m^2*sin(theta(t)) - M*g*l^3*m^2*sin(theta(t)) + 3*g*l^3*m^3*sin(theta(t))*c

In [14]:
qdot = [ diff(H,p) for p in [p1,p2] ]

2-element Array{Sym,1}:
 (J_G*p1 + l^2*m*p1 - l*m*p2*cos(theta(t)))/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(theta(t))^2)
       (M*p2 - l*m*p1*cos(theta(t)) + m*p2)/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(theta(t))^2)

In [15]:
sympy_meth(:octave_code, qdot)

"% Not supported in Octave:\n% theta\n[(J_G.*p1 + l.^2.*m.*p1 - l.*m.*p2.*cos(theta(t)))./(J_G.*M + J_G.*m + M.*l.^2.*m + l.^2.*m.^2.*sin(theta(t)).^2); (M.*p2 - l.*m.*p1.*cos(theta(t)) + m.*p2)./(J_G.*M + J_G.*m + M.*l.^2.*m + l.^2.*m.^2.*sin(theta(t)).^2)]"

That's not good.  Let's define a set of states
$$ z = [x,\theta, p_1,p_2] $$

In [16]:
z1,z2,z3,z4, torque = symbols("z1,z2,z3,z4, torque")
rule1 = Dict( x=> z1, θ=>z2, p1=>z3, p2=>z4, τ=>torque)

## previous error, now fixed:
# by the definition of z, I need to be careful on the ordering:
F = vcat( qdot, pdot ) |> subs(rule1)

4-element Array{Sym,1}:
                                                                                                                                                                                                                                                                                                                                                                              (J_G*z3 + l^2*m*z3 - l*m*z4*cos(z2))/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(z2)^2)
                                                                                                                                                                                                                                                                                                                                                                                    (M*z4 - l*m*z3*cos(z2) + m*z4)/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(z2)^2)
                                                                              

In [17]:
sympy_meth(:octave_code, F) |> println

[(J_G.*z3 + l.^2.*m.*z3 - l.*m.*z4.*cos(z2))./(J_G.*M + J_G.*m + M.*l.^2.*m + l.^2.*m.^2.*sin(z2).^2); (M.*z4 - l.*m.*z3.*cos(z2) + m.*z4)./(J_G.*M + J_G.*m + M.*l.^2.*m + l.^2.*m.^2.*sin(z2).^2); f; 2*l.^2.*m.^2.*(J_G.*M.*g.*l.*m.*cos(z2) + J_G.*g.*l.*m.^2.*cos(z2) + J_G.*z3.^2/2 + M.*g.*l.^3.*m.^2.*cos(z2) + M.*z4.^2/2 - g.*l.^3.*m.^3.*cos(z2).^3 + g.*l.^3.*m.^3.*cos(z2) + l.^2.*m.*z3.^2/2 - l.*m.*z3.*z4.*cos(z2) + m.*z4.^2/2).*sin(z2).*cos(z2)./(J_G.*M + J_G.*m + M.*l.^2.*m + l.^2.*m.^2.*sin(z2).^2).^2 + torque - (-J_G.*M.*g.*l.*m.*sin(z2) - J_G.*g.*l.*m.^2.*sin(z2) - M.*g.*l.^3.*m.^2.*sin(z2) + 3*g.*l.^3.*m.^3.*sin(z2).*cos(z2).^2 - g.*l.^3.*m.^3.*sin(z2) + l.*m.*z3.*z4.*sin(z2))./(J_G.*M + J_G.*m + M.*l.^2.*m + l.^2.*m.^2.*sin(z2).^2)]


In [18]:
sympy_meth(:octave_code, subs(H,rule1) ) |> println

(J_G.*M.*g.*l.*m.*cos(z2) + J_G.*g.*l.*m.^2.*cos(z2) + J_G.*z3.^2/2 + M.*g.*l.^3.*m.^2.*cos(z2) + M.*z4.^2/2 - g.*l.^3.*m.^3.*cos(z2).^3 + g.*l.^3.*m.^3.*cos(z2) + l.^2.*m.*z3.^2/2 - l.*m.*z3.*z4.*cos(z2) + m.*z4.^2/2)./(J_G.*M + J_G.*m + M.*l.^2.*m + l.^2.*m.^2.*sin(z2).^2)


## Linearization

In [22]:
rule_equil = Dict(z1=>0, z2=>0, z3=>0, z4=>0)
A = [ diff(f,z) for f in F, z in [z1,z2,z3,z4] ] |> subs(rule_equil)

4×4 Array{Sym,2}:
 0  …     -l*m/(J_G*M + J_G*m + M*l^2*m)
 0     (M + m)/(J_G*M + J_G*m + M*l^2*m)
 0                                     0
 0                                     0

In [25]:
B = [diff(f,u) for f in F, u in [f,torque]]

4×2 Array{Sym,2}:
 0  0
 0  0
 1  0
 0  1

In [34]:
CM = hcat(B, A*B, A^2*B, A^3*B)
CM = [simplify(x) for x in CM ]

4×8 Array{Sym,2}:
 0  0  …  -g*l^2*m^2*(M + m)/(J_G*M + J_G*m + M*l^2*m)^2
 0  0        g*l*m*(M + m)^2/(J_G*M + J_G*m + M*l^2*m)^2
 1  0                                                  0
 0  1                                                  0

In [43]:
[simplify(a) for a in SymPy.rref(CM)[1] ]

4×8 Array{Sym,2}:
 1  0  0  0  …                                        0
 0  1  0  0                                           0
 0  0  1  0                                           0
 0  0  0  1     g*l*m*(M + m)/(J_G*M + J_G*m + M*l^2*m)